In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

# Dataset Class
class SpeedDataset(Dataset):
    def __init__(self, directory, sequence_length=5):
        self.data = []
        self.sequence_length = sequence_length
        self.preprocess_data(directory)

    def preprocess_data(self, directory):
        for filename in os.listdir(directory):
            if filename.endswith('.txt'):
                speed = float(filename.split('_')[-1].replace('.txt', ''))
                filepath = os.path.join(directory, filename)
                with open(filepath, 'r') as file:
                    track_data = {}
                    for line in file:
                        frame, track_id, x1, y1, x2, y2 = map(float, line.strip().split(',')[:6])
                        if track_id not in track_data:
                            track_data[track_id] = []
                        track_data[track_id].append([x1,y1,x2,y2,(x1+x2)/2, (y1+y2)/2])

                    for track_id, frames in track_data.items():
                        if len(frames) >= self.sequence_length:
                            features = []
                            for i in range(1, len(frames)):
                                current_frame = frames[i]
                                previous_frame = frames[i-1]
                                data_d = []
                                for i in range(4):
                                    data_d.append(current_frame[i])
                                for j in range(4,6):
                                    data_d.append(current_frame[j] - previous_frame[j])
                                features.append(data_d)
                            for i in range(len(features) - self.sequence_length + 1):
                                sequence = features[i:i + self.sequence_length]
                                self.data.append((sequence, speed))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputs, output = self.data[idx]
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(output, dtype=torch.float32)




In [3]:
train_dataset = SpeedDataset('outputs', sequence_length=5)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = SpeedDataset('test', sequence_length=5)
val_loader = DataLoader(val_dataset, batch_size=12, shuffle=False)


In [5]:
train_dataset[8]

(tensor([[ 6.1342e+02,  4.6990e+02,  6.7616e+02,  5.1439e+02,  1.4231e+00,
          -1.6779e+00],
         [ 6.1350e+02,  4.6821e+02,  6.7820e+02,  5.1529e+02,  1.0601e+00,
          -3.9218e-01],
         [ 6.1429e+02,  4.6764e+02,  6.7902e+02,  5.1676e+02,  8.0511e-01,
           4.4865e-01],
         [ 6.1499e+02,  4.6733e+02,  6.8166e+02,  5.1825e+02,  1.6664e+00,
           5.9392e-01],
         [ 6.1576e+02,  4.6674e+02,  6.8360e+02,  5.1932e+02,  1.3587e+00,
           2.3386e-01]]),
 tensor(101.))